In [15]:
import pandas as pd
# Imports CSV
df = pd.read_csv('../raw_data/constituency_raw.csv')
# Reduces CSV to relevant election years
filtered_df = df[df['Election Year'].isin([2005, 2010, 2015, 2017])]
#Creates new index
filtered_df['new_index'] = filtered_df['Election Year'].astype(str) + '_' + filtered_df['Constituency'] + '_' + filtered_df['Boundary Year'].astype(str)
# Pivots table for the first time
pivot_votes_df = filtered_df.pivot_table(
    index=['Election Year', 'Boundary Year', 'Constituency', 'Candidate','Party', 'new_index'],
    values='Votes',
    fill_value=0
).reset_index()

/tmp/ipykernel_57767/408073463.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['new_index'] = filtered_df['Election Year'].astype(str) + '_' + filtered_df['Constituency'] + '_' + filtered_df['Boundary Year'].astype(str)


In [16]:
# Reduces CSV to relevant parties
parties_cleaned = pivot_votes_df.reset_index(drop=False)['Party'].apply(lambda x: x if x in ['British National Party', 'The Brexit Party', 'Conservative', 'Green Party', 'Liberal Democrats', 'Labour', 'Plaid Cymru', 'SNP', 'UK Independence Party (UKIP)'] else 'Other')
pivot_votes_df['Party_cleaned'] = parties_cleaned


In [17]:
#Drops irrelevant columns
pivot_votes_df.drop(columns='Party', inplace=True)
pivot_votes_df.set_index('new_index', inplace=True)


In [18]:
# Makes second pivot
pivot_table = pd.pivot_table(pivot_votes_df,
                             index=['new_index'],
                             columns=['Party_cleaned'],
                             values='Votes',
                             aggfunc='sum',
                             fill_value=0)

In [19]:
# Renames columns to follow our style
rename_dict = {
    'British National Party': 'NAT_ACTUAL_CONS',
    'Conservative': 'CON_ACTUAL_CONS',
    'Green Party': 'GRE_ACTUAL_CONS',
    'Liberal Democrats': 'LIB_ACTUAL_CONS',
    'Labour': 'LAB_ACTUAL_CONS',
    'Plaid Cymru': 'PLC_ACTUAL_CONS',
    'SNP': 'SNP_ACTUAL_CONS',
    'UK Independence Party (UKIP)': 'UKI_ACTUAL_CONS',
    'Other': 'OTH_ACTUAL_CONS'
}
pivot_table.rename(columns=rename_dict, inplace=True)

In [22]:
#Readds relevant columns
pivot_table['Election Year'] = pivot_table.index.str.split('_').str[0].astype(int)
pivot_table['Constituency'] = pivot_table.index.str.split('_').str[1]
pivot_table['Boundary Year'] = pivot_table.index.str.split('_').str[2]

In [24]:
# Reorders columns
columns_reordered = ['Constituency', 'Election Year', 'Boundary Year',
                     'CON_ACTUAL_CONS', 'GRE_ACTUAL_CONS', 'LAB_ACTUAL_CONS',
                     'LIB_ACTUAL_CONS', 'NAT_ACTUAL_CONS', 'OTH_ACTUAL_CONS', 'PLC_ACTUAL_CONS',
                     'SNP_ACTUAL_CONS', 'UKI_ACTUAL_CONS']
df_final = pivot_table[columns_reordered]

In [26]:
# Calculates total votes
df_final['total_votes'] = df_final[['CON_ACTUAL_CONS', 'GRE_ACTUAL_CONS', 'LAB_ACTUAL_CONS', 'LIB_ACTUAL_CONS',
                  'NAT_ACTUAL_CONS', 'OTH_ACTUAL_CONS', 'PLC_ACTUAL_CONS', 'SNP_ACTUAL_CONS',
                  'UKI_ACTUAL_CONS']].sum(axis=1)

/tmp/ipykernel_57767/1285122342.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['total_votes'] = df_final[['CON_ACTUAL_CONS', 'GRE_ACTUAL_CONS', 'LAB_ACTUAL_CONS', 'LIB_ACTUAL_CONS',


In [29]:
# Switches to percentage
df_percentage = df_final[['CON_ACTUAL_CONS', 'GRE_ACTUAL_CONS', 'LAB_ACTUAL_CONS', 'LIB_ACTUAL_CONS', 'NAT_ACTUAL_CONS',
                    'OTH_ACTUAL_CONS', 'PLC_ACTUAL_CONS', 'SNP_ACTUAL_CONS',
                    'UKI_ACTUAL_CONS']].div(df_final['total_votes'], axis=0) * 100

In [30]:
df_result = pd.concat([df_final[['Election Year', 'Constituency']], df_percentage], axis=1)


In [31]:
df_result.rename(columns={'Election Year': 'election_year'}, inplace=True)

In [32]:
df_result

Party_cleaned,election_year,Constituency,CON_ACTUAL_CONS,GRE_ACTUAL_CONS,LAB_ACTUAL_CONS,LIB_ACTUAL_CONS,NAT_ACTUAL_CONS,OTH_ACTUAL_CONS,PLC_ACTUAL_CONS,SNP_ACTUAL_CONS,UKI_ACTUAL_CONS
new_index,,,,,,,,,,,
2005_Aberavon_2005,2005,Aberavon,10.178049,1.694127,60.048499,13.752325,0.0,2.551156,11.775844,0.000000,0.000000
2005_Aberdeen North_2005,2005,Aberdeen North,9.433859,0.000000,42.466015,23.917672,0.0,1.886226,0.000000,22.296228,0.000000
2005_Aberdeen South_2005,2005,Aberdeen South,17.140386,0.000000,36.693016,33.454266,0.0,2.813484,0.000000,9.898849,0.000000
2005_Aberdeenshire West and Kincardine_2005,2005,Aberdeenshire West and Kincardine,28.366308,0.000000,13.133884,46.304745,0.0,0.910008,0.000000,11.285056,0.000000
2005_Airdrie and Shotts_2005,2005,Airdrie and Shotts,9.864889,0.000000,59.014416,11.436154,0.0,3.145546,0.000000,16.538995,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2017_Yeovil_2010,2017,Yeovil,54.489597,1.770925,12.487375,29.705070,0.0,1.547034,0.000000,0.000000,0.000000
2017_Ynys Mon_2010,2017,Ynys Mon,27.789226,0.000000,41.863141,1.281880,0.0,0.000000,27.395831,0.000000,1.669923
2017_York Central_2010,2017,York Central,30.174427,0.000000,65.163502,4.662071,0.0,0.000000,0.000000,0.000000,0.000000


In [33]:
# Imports national election results to concatenante
election_results_df = pd.read_csv('../processed_data/election_results_dataframe.csv')
election_results_df.rename(columns={'Year': 'election_year'}, inplace=True)

In [34]:
election_results_df.rename(columns={'Year': 'election_year'}, inplace=True)

In [35]:
# Merges national onto constituency election results
cons_result_comp = df_result.merge(election_results_df, on='election_year', how='left')

In [36]:
# Renames national result columns
rename_dict2 = {
    'BRX_ACTUAL_PERCENTAGE': 'BRX_NAT_RESULT',
    'CON_ACTUAL_PERCENTAGE': 'CON_NAT_RESULT',
    'GRE_ACTUAL_PERCENTAGE': 'GRE_NAT_RESULT',
    'LIB_ACTUAL_PERCENTAGE': 'LIB_NAT_RESULT',
    'LABOUR_ACTUAL_PERCENTAGE': 'LAB_NAT_RESULT',
    'NAT_ACTUAL_PERCENTAGE': 'NAT_NAT_RESULT',
    'PLC_ACTUAL_PERCENTAGE': 'PLC_NAT_RESULT',
    'SNP_ACTUAL_PERCENTAGE': 'SNP_NAT_RESULT',
    'UKI_ACTUAL_PERCENTAGE': 'UKI_NAT_RESULT',
    'OTH_PERCENTAGE': 'OTH_NAT_RESULT'
}
cons_result_comp.rename(columns=rename_dict2, inplace=True)


In [37]:
# Drops and renames columns
cons_result_comp.drop(columns=['Geography', 'Country'], inplace=True)
cons_result_comp.rename(columns={'CON_ACT_CONS': 'CON_ACTUAL_CONS'}, inplace=True)
cons_result_comp.rename(columns={'BRX_ACT_CONS': 'BRX_ACTUAL_CONS'}, inplace=True)

In [39]:
# Calculates bias score
for index, row in cons_result_comp.iterrows():
    bias_scores = {}
    # Loop through each party
    for party in ['CON', 'GRE', 'LAB', 'LIB', 'NAT', 'OTH', 'PLC', 'SNP', 'UKI']:
        # Calculate the absolute difference between the constituency result and national result for each party
        bias_scores[party] = (row[f'{party}_ACTUAL_CONS'] - row[f'{party}_NAT_RESULT'])
    # Assign the bias scores to the respective columns
    for party, score in bias_scores.items():
        cons_result_comp.loc[index, f'{party}_Bias_Score'] = score

In [40]:
# Filters datafra,e
biases_df = cons_result_comp.filter(like='Bias')

In [41]:
# Calculates bias per constituency
average_biases_per_constituency = biases_df.groupby(cons_result_comp['Constituency']).mean()

In [42]:
average_biases_per_constituency

,CON_Bias_Score,GRE_Bias_Score,LAB_Bias_Score,LIB_Bias_Score,NAT_Bias_Score,OTH_Bias_Score,PLC_Bias_Score,SNP_Bias_Score,UKI_Bias_Score
Constituency,,,,,,,,,
Aberavon,-23.383095,-0.776212,23.593507,-6.011279,0.372713,-0.574823,9.122838,-2.736884,0.393236
Aberconwy,2.237431,-1.231315,-1.362923,-3.805160,-0.640285,-3.313033,12.584204,-3.142973,-1.325945
Aberdeen North,-22.748165,-1.763618,2.054465,-2.132780,-0.236639,-2.465155,-0.575947,32.822584,-4.954744
Aberdeen South,-11.186185,-1.763618,-3.520760,3.013810,-0.350401,-2.101086,-0.575947,20.977045,-4.492859
Aberdeenshire West and Kincardine,-3.062077,-1.763618,-23.074161,13.617842,-0.373945,-3.000303,-0.575947,22.511700,-4.279491
...,...,...,...,...,...,...,...,...,...
Ynys Mon,-16.271884,-1.763618,1.588433,-10.628608,-0.657716,1.987405,28.224095,-2.736884,0.258776
York Central,-10.215902,2.511350,16.028050,-0.155499,0.199448,-2.950186,-0.553252,-3.142973,-1.721036
York Outer,9.339253,0.181736,-6.964496,6.574726,-0.042412,-3.465428,-0.553252,-3.142973,-1.927154


In [43]:
 average_biases_per_constituency.to_csv('../processed_data/removes_2019_constituency_bias_dataframe.csv', index=True)